Code required to authenticate with Earth Engine, originally taken from: https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/authorize_notebook_server.ipynb

In [1]:
# Code to check the IPython Widgets library.
try:
    import ipywidgets
except ImportError:
    print('The IPython Widgets library is not available on this server.\n'
                'Please see https://github.com/jupyter-widgets/ipywidgets '
                'for information on installing the library.')
    raise
print('The IPython Widgets library (version {0}) is available on this server.'.format(
    ipywidgets.__version__
))

The IPython Widgets library (version 7.1.1) is available on this server.


In [2]:
# Code to check the Earth Engine API library.
try:
    import ee
except ImportError:
    print('The Earth Engine Python API library is not available on this server.\n'
                'Please see https://developers.google.com/earth-engine/python_install '
                'for information on installing the library.')
    raise
print('The Earth Engine Python API (version {0}) is available on this server.'.format(
    ee.__version__
))

The Earth Engine Python API (version 0.1.137) is available on this server.


In [3]:
# Code to check if authorized to access Earth Engine.
import io
import os
import urllib
from IPython import display

# Define layouts used by the form.
row_wide_layout = ipywidgets.Layout(flex_flow="row nowrap", align_items="center", width="100%")
column_wide_layout = ipywidgets.Layout(flex_flow="column nowrap", align_items="center", width="100%")
column_auto_layout = ipywidgets.Layout(flex_flow="column nowrap", align_items="center", width="auto")

form_definition = {'form': None}
response_box = ipywidgets.HTML('')

def isAuthorized():
    try:
        ee.Initialize()
        test = ee.Image(0).getInfo()
    except:
        return False
    return True

def ShowForm(auth_status_button, instructions):
    """Show a form to the user."""
    form_definition['form'] = ipywidgets.VBox([
        auth_status_button,
        instructions,
        ipywidgets.VBox([response_box], layout=row_wide_layout)
    ], layout=column_wide_layout)
    display.display(form_definition.get('form'))

def ShowAuthorizedForm():
    """Show a form for a server that is currently authorized to access Earth Engine."""
    def revoke_credentials(sender):
        credentials = ee.oauth.get_credentials_path()
        if os.path.exists(credentials):
            os.remove(credentials)
        response_box.value = ''
        Init()
    
    auth_status_button = ipywidgets.Button(
        layout=column_wide_layout,
        disabled=True,
        description='The server is authorized to access Earth Engine',
        button_style='success',
        icon='check'
    )
    
    instructions = ipywidgets.Button(
        layout = row_wide_layout,
        description = 'Click here to revoke authorization',
        disabled = False,
    )
    instructions.on_click(revoke_credentials)
    
    ShowForm(auth_status_button, instructions)

def ShowUnauthorizedForm():
    """Show a form for a server that is not currently authorized to access Earth Engine."""
    def save_credentials(sender):
        try:
            # Save the credentials text, and clear the textbox.
            textbox_value = get_auth_textbox.value
            get_auth_textbox.value = ''
            
            token = ee.oauth.request_token(textbox_value.strip())
            ee.oauth.write_token(token)
            if isAuthorized():
                response_box.value = ''
                Init()
            else:
                response_box.value = '<font color="red">{0}</font>'.format(
                    'The account was authenticated, but does not have permission to access Earth Engine.'
                )
        except Exception as e:
            response_box.value = '<font color="red">{0}</font>'.format(e)
    
    get_auth_textbox = ipywidgets.Text(
        placeholder='Paste authorization code here',
        description='Authentication Code:',
    )
    get_auth_textbox.on_submit(save_credentials)

    auth_status_button = ipywidgets.Button(
        layout=column_wide_layout,
        button_style='danger',
        description='The server is not authorized to access Earth Engine',
        disabled=True
    )
        
    auth_link = ipywidgets.HTML(
        '<a href="{url}" target="auth">Open Authentication Tab</a><br/>'
        .format(url=ee.oauth.get_authorization_url()
        )
    )
        
    instructions = ipywidgets.VBox(
        [
            ipywidgets.HTML(
                'Click on the link below to start the authentication and authorization process. '
                'Once you have received an authorization code, paste it in the box below and press return.'
            ),
            auth_link,
            get_auth_textbox
        ],
        layout=column_auto_layout
    )
    
    ShowForm(auth_status_button, instructions)
    
def Init():
    # If a form is currently displayed, close it.
    if form_definition.get('form'):
        form_definition['form'].close()
    # Display the appropriate form according to whether the server is authorized.
    if isAuthorized():
        ShowAuthorizedForm()
    else:
        ShowUnauthorizedForm()
        
Init()

VBox(children=(Button(button_style='success', description='The server is authorized to access Earth Engine', disabled=True, icon='check', layout=Layout(align_items='center', flex_flow='column nowrap', width='100%'), style=ButtonStyle()), Button(description='Click here to revoke authorization', layout=Layout(align_items='center', flex_flow='row nowrap', width='100%'), style=ButtonStyle()), VBox(children=(HTML(value=''),), layout=Layout(align_items='center', flex_flow='row nowrap', width='100%'))), layout=Layout(align_items='center', flex_flow='column nowrap', width='100%'))